In [2]:
from constant import openai_key
import sys
import os
from pypdf import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

os.environ["OPENAI_API_KEY"] = openai_key
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from typing_extensions import Concatenate

In [3]:
pdfreader = PdfReader("Trustllm.pdf")

In [4]:
raw_text = ""
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [5]:
raw_text

'TRUST LLM: T RUSTWORTHINESS IN LARGE LANGUAGE MODELS\nLichao Sun1∗†Yue Huang1∗† ‡Haoran Wang2∗Siyuan Wu1∗‡Qihui Zhang1∗‡Chujie Gao1∗‡\nYixin Huang3∗Wenhan Lyu4∗Yixuan Zhang4∗Xiner Li5∗Zhengliang Liu6∗\nYixin Liu1∗Yijue Wang7∗Zhikun Zhang8∗Bhavya Kailkhura9Caiming Xiong10\nChao Zhang11Chaowei Xiao12Chunyuan Li13Eric Xing14,40Furong Huang15\nHao Liu16Heng Ji17Hongyi Wang14Huan Zhang17Huaxiu Yao18Manolis Kellis19\nMarinka Zitnik20Meng Jiang21Mohit Bansal18James Zou8Jian Pei22\nJian Liu23Jianfeng Gao13Jiawei Han17Jieyu Zhao24Jiliang Tang25\nJindong Wang26John Mitchell8Kai Shu2Kaidi Xu27Kai-Wei Chang28\nLifang He1Lifu Huang29Michael Backes30Neil Zhenqiang Gong22\nPhilip S. Yu31Pin-Yu Chen32Quanquan Gu28Ran Xu10Rex Ying33Shuiwang Ji5\nSuman Jana34Tianlong Chen18Tianming Liu6Tianyi Zhou15Willian Wang35\nXiang Li36Xiangliang Zhang21Xiao Wang37Xing Xie26Xun Chen7Xuyu Wang38\nYan Liu24Yanfang Ye21Yinzhi Cao39Yue Zhao24\n1Lehigh University2Illinois Institute of Technology3Institut Polytechnique 

In [6]:
text_splitter = CharacterTextSplitter(
    chunk_size = 800,
    chunk_overlap = 200,
    separator="\n",
    length_function = len
)

texts = text_splitter.split_text(raw_text)

In [7]:
texts

['TRUST LLM: T RUSTWORTHINESS IN LARGE LANGUAGE MODELS\nLichao Sun1∗†Yue Huang1∗† ‡Haoran Wang2∗Siyuan Wu1∗‡Qihui Zhang1∗‡Chujie Gao1∗‡\nYixin Huang3∗Wenhan Lyu4∗Yixuan Zhang4∗Xiner Li5∗Zhengliang Liu6∗\nYixin Liu1∗Yijue Wang7∗Zhikun Zhang8∗Bhavya Kailkhura9Caiming Xiong10\nChao Zhang11Chaowei Xiao12Chunyuan Li13Eric Xing14,40Furong Huang15\nHao Liu16Heng Ji17Hongyi Wang14Huan Zhang17Huaxiu Yao18Manolis Kellis19\nMarinka Zitnik20Meng Jiang21Mohit Bansal18James Zou8Jian Pei22\nJian Liu23Jianfeng Gao13Jiawei Han17Jieyu Zhao24Jiliang Tang25\nJindong Wang26John Mitchell8Kai Shu2Kaidi Xu27Kai-Wei Chang28\nLifang He1Lifu Huang29Michael Backes30Neil Zhenqiang Gong22\nPhilip S. Yu31Pin-Yu Chen32Quanquan Gu28Ran Xu10Rex Ying33Shuiwang Ji5\nSuman Jana34Tianlong Chen18Tianming Liu6Tianyi Zhou15Willian Wang35',
 'Lifang He1Lifu Huang29Michael Backes30Neil Zhenqiang Gong22\nPhilip S. Yu31Pin-Yu Chen32Quanquan Gu28Ran Xu10Rex Ying33Shuiwang Ji5\nSuman Jana34Tianlong Chen18Tianming Liu6Tianyi Zhou15W

In [8]:
len(texts)

712

In [9]:
embedding = OpenAIEmbeddings()



c:\Users\Aai\anaconda3\envs\pytorch\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [10]:
!rm -rf ./docs/chroma  # remove old database files if any
persist_directory = 'docs/chroma/'

'rm' is not recognized as an internal or external command,
operable program or batch file.


In [11]:
document_search = FAISS.from_texts(texts,embedding)

In [12]:
document_search

In [13]:
chain = load_qa_chain(OpenAI(),chain_type="stuff")

c:\Users\Aai\anaconda3\envs\pytorch\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [14]:
query = "which datasets was used in Integrating External Knowledge?"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs,question=query)

c:\Users\Aai\anaconda3\envs\pytorch\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


' Climate-FEVER, SQuAD2.0, CODAH, and HotpotQA were used as datasets for Integrating External Knowledge.'

In [15]:
query = "In table 7 which model perfomed well"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs,question=query)

' GPT-4 performed well in table 7, as it had the highest F1 score and was highlighted in green.'

In [16]:
query = "which datasets was used in Misinformation Generation"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs,question=query)

' The datasets used in Misinformation Generation are SQUAD2.0, CODAH, HOTPOT QA, ADVERSARIAL QA, and CLIMATE-FEVER.'

In [17]:
query = "what evaluation methods was used in Misinformation Generation"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs,question=query)

" The evaluation methods used in Misinformation Generation were assessing the LLMs' inclination to generate misinformation and testing their propensity to hallucinate across multiple tasks."